#転移学習を実装してみよう！

In [0]:
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, BatchNormalization
from keras.layers import Add, Activation, Flatten, Dense, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import cifar10
from keras.utils import to_categorical

import keras
from keras.applications import VGG16

# データの読み込み
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0
Y_train, Y_test = to_categorical(y_train), to_categorical(y_test)

In [0]:
classes = 10

model = VGG16(include_top=False, input_shape=X_train.shape[1:], weights='imagenet')

# 全結合層を付ける
layer = GlobalAveragePooling2D()(model.output)
out = Dense(classes, activation='softmax')(layer)
model = Model(inputs=model.input, outputs=out)

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.0001, amsgrad=True),
              metrics=['accuracy'])
    
# Data Augmentation
datagen = ImageDataGenerator(rotation_range=20,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             horizontal_flip=True)

datagen.fit(X_train)

#cnnの学習
hist = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=128),
                           steps_per_epoch=len(X_train) / 128,
                           validation_data=(X_test, Y_test),
                           epochs=20)

#結果描画
plt.figure()               
plt.plot(hist.history['acc'],label="train_acc")
plt.plot(hist.history['val_acc'],label="val_acc")
plt.legend(loc="lower right")
plt.show()

plt.figure()               
plt.plot(hist.history['loss'],label="train_loss")
plt.plot(hist.history['val_loss'],label="val_loss")
plt.legend()
plt.show()

In [0]:
#cifar10のラベル
labels = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck']

In [0]:
test_NO = 6000

plt.imshow(X_test[test_NO])
plt.title(labels[y_test[test_NO][0]])
plt.show()

In [0]:
fig = np.expand_dims(X_test[test_NO],axis=0)
result = model.predict(fig)
result = result.reshape(-1)

for (label, score) in zip(labels, result):
    print(label+ " : " + str(score))

#自前の画像で学習させよう

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd /content/drive//My Drive/data

In [0]:
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, BatchNormalization
from keras.layers import Add, Activation, Flatten, Dense, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from PIL import Image
import cv2

import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import cifar10
from keras.utils import to_categorical

import keras
from keras.applications import VGG16
import glob

def load_img(dir_):
    result = []
    filenames = glob.glob(dir_ + "*.jpg")
    for name in filenames:
        img = Image.open(name)
        img = img_to_array(img)
        img = cv2.resize(img,(96,96))/255
        result.append(img)

    return np.array(result)

In [0]:
class0_train = load_img("train/class0/")
class1_train = load_img("train/class1/")
class0_test = load_img("test/class0/")
class1_test = load_img("test/class1/")

X_train = np.vstack((class0_train, class1_train))
X_test = np.vstack((class0_test, class1_test))

y_train = np.zeros(len(class0_train))
y_train = np.hstack((y_train, np.ones(len(class1_train))))
Y_train = to_categorical(y_train)

y_test = np.zeros(len(class0_test))
y_test = np.hstack((y_test, np.ones(len(class1_test))))
Y_test = to_categorical(y_test)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

In [0]:
classes = 2

model = VGG16(include_top=False, input_shape=X_train.shape[1:], weights='imagenet')

# 全結合層を付ける
layer = GlobalAveragePooling2D()(model.output)
out = Dense(classes, activation='softmax')(layer)
model = Model(inputs=model.input, outputs=out)

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.0001, amsgrad=True),
              metrics=['accuracy'])
    
# Data Augmentation
datagen = ImageDataGenerator(rotation_range=20,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             horizontal_flip=True)

datagen.fit(X_train)

#cnnの学習
hist = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=8),
                           steps_per_epoch=len(X_train) / 8,
                           validation_data=(X_test, Y_test),
                           epochs=20)

#結果描画
plt.figure()               
plt.plot(hist.history['acc'],label="train_acc")
plt.plot(hist.history['val_acc'],label="val_acc")
plt.legend(loc="lower right")
plt.show()

plt.figure()               
plt.plot(hist.history['loss'],label="train_loss")
plt.plot(hist.history['val_loss'],label="val_loss")
plt.legend()
plt.show()

In [0]:
#grad-CAM
import keras.backend as K
import cv2
from keras.preprocessing.image import array_to_img

def GradCam(model, x, layer_name):

    X = np.expand_dims(x,axis=0)
    
    # 前処理
    predictions = model.predict(X)
    class_idx = np.argmax(predictions[0])
    class_output = model.output[:, class_idx]
    
    # 勾配を取得
    conv_output = model.get_layer(layer_name).output   # layer_nameのレイヤーのアウトプット
    grads = K.gradients(class_output, conv_output)[0]  # gradients(loss, variables) で、variablesのlossに関しての勾配を返す
    gradient_function = K.function([model.input], [conv_output, grads])  # model.inputを入力すると、conv_outputとgradsを出力する関数

    output, grads_val = gradient_function([X])
    output, grads_val = output[0], grads_val[0]

    # 重みを平均化して、レイヤーのアウトプットに乗じる
    weights = np.mean(grads_val, axis=(0, 1))
    cam = np.dot(output, weights)

    # ヒートマップにして合成
    cam = cv2.resize(cam, (x.shape[1], x.shape[0]), cv2.INTER_LINEAR)
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()
    
    jetcam = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)  # モノクロ画像に疑似的に色をつける
    jetcam = cv2.cvtColor(jetcam, cv2.COLOR_BGR2RGB)  # 色をRGBに変換
    jetcam = (np.float32(jetcam) + x*255 / 2)   # もとの画像に合成

    plt.figure(figsize=(8,4))
    plt.subplot(1,2,1)
    plt.imshow(x)
    plt.title("predict:"+labels[class_idx])

    plt.subplot(1,2,2)
    plt.imshow(array_to_img(jetcam))
    plt.title("GradCAM")
    plt.show()

In [0]:
# ラベル
labels = [
    'Gyoku',
    'Fu']

In [0]:
test_NO = 8

plt.imshow(X_test[test_NO])
plt.title(labels[int(y_test[test_NO])])
plt.show()

In [0]:
fig = np.expand_dims(X_test[test_NO],axis=0)
result = model.predict(fig)
result = result.reshape(-1)

for (label, score) in zip(labels, result):
    print(label+ " : " + str(score))

In [0]:
img = GradCam(model, fig[0], "block5_pool")